In [1]:
pip install tweepy



*   Input: The script will ask the user to input a tweet URL.
*   Tweet ID Extraction: It will extract the tweet ID using regular expressions (extract_tweet_id function).

*   Fetching Comments: The get_tweet_comments function fetches the comments from the tweet.
*   Excel File Generation: The comments are saved into an Excel file using pandas and the file is then ready for download.

*   Google Colab Download: The files.download function is used to allow the user to download the Excel file with the comments.



In [3]:
import re
import tweepy
import pandas as pd
from google.colab import files

# Set up Tweepy client with your BEARER_TOKEN
BEARER_TOKEN = "*********************************************"

client = tweepy.Client(bearer_token=BEARER_TOKEN)

# Function to extract tweet ID from URL
def extract_tweet_id(url):
    match = re.search(r"status/(\d+)", url)
    return match.group(1) if match else None

# Function to get comments of the tweet
def get_tweet_comments(tweet_id, max_results=100):
    try:
        query = f"conversation_id:{tweet_id}"
        all_comments = []
        next_token = None

        while True:
            response = client.search_recent_tweets(
                query=query,
                tweet_fields=['author_id', 'conversation_id', 'text'],
                max_results=max_results,
                next_token=next_token
            )
            if response.data:
                all_comments.extend([{'username': tweet.author_id, 'text': tweet.text} for tweet in response.data])

            next_token = response.meta.get('next_token') if 'meta' in response else None
            if not next_token:
                break

        return all_comments
    except Exception as e:
        print(f"Error: {e}")
        return []

# Function to download comments as an Excel file
def download_comments_excel(tweet_url):
    tweet_id = extract_tweet_id(tweet_url)

    if tweet_id:
        comments = get_tweet_comments(tweet_id)

        if comments:
            # Convert comments to a DataFrame
            df = pd.DataFrame(comments)
            # Save to Excel
            file_name = "tweet_comments.xlsx"
            df.to_excel(file_name, index=False)

            # Provide download link
            files.download(file_name)
            print(f"Download link for comments file: {file_name}")
        else:
            print("No comments found for the tweet.")
    else:
        print("Invalid tweet URL. Please ensure it is a valid tweet URL.")

# Input the tweet URL
tweet_url = input("Enter the tweet URL: ")
download_comments_excel(tweet_url)


Enter the tweet URL: https://x.com/SportsCenter/status/1878690015963680946?t=evOkjNLJ8Ky2YlcOTRSPAA&s=08
Error: 401 Unauthorized
Unauthorized
No comments found for the tweet.
